[View in Colaboratory](https://colab.research.google.com/github/jomarlin/learningisfun/blob/master/sui_task.ipynb)

In [0]:
!pip install vk_api
import vk_api
import pandas as pd
import google.colab.files as files
import re
import numpy as np
from Collections import Counter
import pymorphy2


[ТОКЕН](https://oauth.vk.com/authorize?client_id=5310660&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends,status,wall,groups,offline&response_type=token)

In [0]:
TOKEN = "" #сюда вставлять полученный по ссылке выше токен VK 
vk_session = vk_api.VkApi(app_id=36136488, token=TOKEN) #id приложения, созданного преподавателем, необходимо для получения токена, без которого часть методов не будет работать
vk = vk_session.get_api()

In [0]:
offset=0
while True:
  followers = vk.groups.getMembers(group_id=XXX, fields="bdate", count=1000, offset=offset) #выкачивает дату рождения всех юзеров в группе с id XXX по 1000 за раз (иначе api vk заблочит запрос)
  all_followers = all_followers + followers['items']
  offset = offset + 1000
  if len(all_followers) == followers['count']:
    break

In [0]:
table = pd.DataFrame( all_followers ) #делает массив пользователей датафреймом
bday = table['bdate'][0]

def age(bday):
  if type(bday) == str and len(bday) > 7:
    return 2018 - int(bday[-4:])
  else:
    return None #вычисляет возраст юзера на основе даты рождения
  
table['age'] = table['bdate'].apply(age) #теперь в таблице есть столбец с возрастом

filtered_table = table[table['age'] < 95]
filtered_table.hist() #смотрим гистограмму по возрасту подписчиков сообщества 




In [0]:
#выкачиваем тексты постов про самоубийство со стены сообщества
sui_text = []
start = ''

while True:
  sui_text = vk.wall.search(owner_id = '-XXX', query="suicide самоубийство ", extended = 1, fields="bdate,sex,about,interests", count=200, start_from=start) #собираем сами тексты и в отдельный массив информацию о юзерах
  person1 = person1 + posts['items']
  offset = offset + 100
  if 'next_from' not in posts:
    break
  start = posts['next_from']

In [0]:
sui_text = sui_text[]['text']

import re

def words(text):
  text = text.lower()
  text = text.replace('ё', 'е')
  text = re.sub('[^а-я ]', '', text)
  text = re.sub(' +', ' ', text)  
  text = text.strip()
  return text.split(' ')

words(sui_text) #обрабатывает текст - приводит к lowercase, убирает лишние символы, сплитит по каждому слову


In [0]:
stopwords = """быть
а
вот
вы
да
еще
и
как
мы
не
нет
он
за
о
они
от
с
сказать
только
у
этот
большой
в
все
говорить
для
же
из
который
на
него
них
один
оно
ото
свой
та
тот
что
я
бы
весь
всей
год
до
знать
к
мочь
наш
нее
но
она
оный
по
себя
такой
ты
это"""

stopwords_list = stopwords.split("\n")
stopwords_list #список стоп-слов, которые будут вырезаны из текстов, чтобы не мешать обработке. список был взят из примера на хабре

In [0]:
def clean_text(text):
  words_list = words(text)
  norm_list = []
  for word in words_list:
    morph_word = morph.parse(word)[0]
    if morph_word.tag.POS not in ['NPRO', 'PREP', 'CONJ', 'PRCL']:
      norm = morph_word.normal_form
      if norm not in stopwords_list:
        norm_list.append(norm)
  return norm_list

clean_text(sui_text) #текст очищается от стоп-слов, возвращается массив со словами, приведенными в начальную форму

In [0]:
suicide_counter = Counter() #присваивает каунтер текстам про самоубийство

for post in sui_text:
  post_words = clean_text(post['text'])
  suicide_counter.update(post_words)
  



In [0]:
print(suicide_counter.most_common(10)) #выводит самые часто встречающиеся слова в текстах про самоубийство

